<a href="https://colab.research.google.com/github/Pradeep0710/MachineLearning/blob/master/cyrpto_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
! pip install kaggle

In [8]:
! mkdir ~/.kaggle

In [11]:
pwd

'/content'

In [12]:
! cp kaggle.json ~/.kaggle/

In [13]:
! chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -c g-research-crypto-forecasting

  0% 0.00/660k [00:00<?, ?B/s]
100% 660k/660k [00:00<00:00, 45.0MB/s]
  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 54.3kB/s]
 99% 1.01G/1.02G [00:07<00:00, 175MB/s]
100% 1.02G/1.02G [00:07<00:00, 150MB/s]
  0% 0.00/406 [00:00<?, ?B/s]
100% 406/406 [00:00<00:00, 383kB/s]
  0% 0.00/5.78k [00:00<?, ?B/s]
100% 5.78k/5.78k [00:00<00:00, 5.78MB/s]
 88% 83.0M/94.2M [00:00<00:00, 122MB/s]
100% 94.2M/94.2M [00:00<00:00, 129MB/s]
  0% 0.00/444 [00:00<?, ?B/s]
100% 444/444 [00:00<00:00, 1.30MB/s]


In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [16]:
ls -ltr

total 1166908
drwxr-xr-x 1 root root       4096 Jan  7 14:33 sample_data/
drwx------ 5 root root       4096 Jan 12 04:29 drive/
-rw-r--r-- 1 root root         74 Jan 12 05:00 kaggle.json
-rw-r--r-- 1 root root     675720 Jan 12 05:01 competition.cpython-37m-x86_64-linux-gnu.so
-rw-r--r-- 1 root root         59 Jan 12 05:01 __init__.py
-rw-r--r-- 1 root root 1095397750 Jan 12 05:01 train.csv.zip
-rw-r--r-- 1 root root        406 Jan 12 05:01 example_sample_submission.csv
-rw-r--r-- 1 root root       5923 Jan 12 05:01 example_test.csv
-rw-r--r-- 1 root root   98798822 Jan 12 05:01 supplemental_train.csv.zip
-rw-r--r-- 1 root root        444 Jan 12 05:01 asset_details.csv


In [17]:
pwd

'/content'

In [29]:
cd /root

/root


In [39]:
mkdir ~/.kaggle/input

In [40]:
!cp /content/* ~/.kaggle/input

cp: -r not specified; omitting directory '/content/sample_data'


In [45]:
import os
import gc
import traceback
import numpy as np
import pandas as pd
import seaborn as sns
#import gresearch_crypto
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
#import pandas as pd, numpy as np
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
pd.set_option('display.max_columns', None)

In [46]:
DEVICE = "TPU" #or "GPU"

SEED = 42

EPOCHS = 25
DEBUG = True
N_ASSETS = 14
WINDOW_SIZE = 15
BATCH_SIZE = 1024
PCT_VALIDATION = 10 # last 10% of the data are used as validation set

In [47]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None
    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except: print("failed to initialize TPU")
    else: DEVICE = "GPU"

if DEVICE != "TPU": strategy = tf.distribute.get_strategy()
if DEVICE == "GPU": print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

connecting to TPU...
Num GPUs Available:  0


In [49]:
pip install datatable

     |████████████████████████████████| 96.9 MB 104 kB/s 


In [50]:
import datatable as dt

In [18]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [58]:
!cd /c

NameError: ignored

In [19]:
!unzip supplemental_train.csv.zip


Archive:  supplemental_train.csv.zip
  inflating: supplemental_train.csv  


In [76]:
!cp supplemental_train.csv /content/

In [77]:
# Uncomment to load the original csv [slower]
orig_df_train = pd.read_csv('/content/' + 'train.csv') 
supp_df_train = pd.read_csv('/content/' + 'supplemental_train.csv')
df_asset_details = pd.read_csv('/content/'  + 'asset_details.csv').sort_values("Asset_ID")

In [80]:
df_asset_details.head()

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin


In [86]:
orig_df_train.isna().any(axis=1)

0           False
1           False
2           False
3           False
4           False
            ...  
24236801     True
24236802     True
24236803     True
24236804     True
24236805     True
Length: 24236806, dtype: bool

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from collections import deque
import random
import time
import math

In [24]:
data_folder = "/content/"

In [25]:
!ls $data_folder

asset_details.csv			     kaggle.json
competition.cpython-37m-x86_64-linux-gnu.so  sample_data
drive					     supplemental_train.csv
example_sample_submission.csv		     supplemental_train.csv.zip
example_test.csv			     train.csv
__init__.py				     train.csv.zip


In [26]:
crypto_df = pd.read_csv(data_folder + 'train.csv')
asset_details = pd.read_csv(data_folder + 'asset_details.csv')

In [27]:
# Adding Crypto symbols to the dataframes for easy reference
asset_symbols_map = {
    0: 'BNB',
    1: 'BTC',
    2: 'BCH',
    3: 'ADA',
    4: 'DOGE',
    5: 'EOS.IO',
    6: 'ETH',
    7: 'ETC',
    8: 'IOTA',
    9: 'LTC',
    10: 'MKR',
    11: 'XMR',
    12: 'XLM',
    13: 'TRON'
}
crypto_df['Symbol'] = crypto_df.Asset_ID.map(asset_symbols_map)
asset_details['Symbol'] = asset_details.Asset_ID.map(asset_symbols_map)

In [28]:
crypto_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,Symbol
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218,BCH
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,BNB
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643,BTC
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922,EOS.IO
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264,ETC
...,...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN,LTC
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN,MKR
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN,TRON
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN,XLM


In [29]:
# how many preceeding periods used to predict future prices
SEQ_LEN = 60

# how many periods forward are we predicting
FUTURE_PERIOD_PREDICT = 3 

# which crypto to predict
CRYPTO_TO_PREDICT = "BTC"


In [30]:
def classify(current, future):
    """
    Compare current vs future crypto price, 
    returns 1 if future price is greater or equal
    with current price, else returns 0.
    """
    if future >= current:
        return 1
    return 0

def preprocess(df):
    """
    Wrapper function to preprocess df for ML input.
    
    Args:
        df: dataframe consisting crypto data (features and target)
        
    Returns:
        x_train, y_train, x_test, y_test: train and test dfs
            ready for ML modeling.
    """
    # Normalizing each features
    df_norm = df.apply(lambda col: scale_normalize(col))
    
    # Creating sequential data
    seq_data = create_sequential_data(df_norm)    
    
    # Train test split
    x_train, y_train, x_test, y_test = train_test_split_seq(seq_data)
    return x_train, y_train, x_test, y_test

def scale_normalize(col):
    """
    Calculates the `pct_change` of a pandas series
    and normalize the values.
    
    Args:
        col: pandas series of float numbers (crypto prices)
        
    Returns:
        pandas series of processed columns
    """
    if 'Buy' in col.name:
        # Skipping preprocessing for `Buy` column
        # Removing first row to sync with preprocessed columns
        return col[1:] 
    else:
        col_processed = col.pct_change()
        col_processed.dropna(inplace=True)
        
        col_processed = preprocessing.scale(col_processed)
        return col_processed
    
def create_sequential_data(df, sequence_length=SEQ_LEN):
    """
    Given a `df`, generate a list (sequence) of smaller `dfs`,
    with each one being a slice of the original `df` of length
    `sequence_length`
    
    Args:
        df: pandas DataFrame to generate the seq data from
        sequence_length: the number of rows in each slice
        
    Returns:
        List of numpy arrays containing features and target
            i.e. [(feature1, target1), (feature2, target2), ..]
    """
    seq_data = []
    
    for row_idx in range(0, len(df)-sequence_length):
        seq_data.append(
            [df.iloc[row_idx:row_idx+sequence_length, :-1],
             df.iloc[row_idx+sequence_length, -1]]
        )
    
    return seq_data

def train_test_split_seq(seq_data, test_frac=0.1):
    """
    Custom function to split the sequential data to 
    train and test sets for ML modeling.
    
    Args:
        seq_data: list of np.arrays containing (feature, target)
        test_frac: proportion of data to assign as test set
    """
    test_size = math.ceil(len(seq_data)*test_frac)
    
    train_seq = seq_data[:-test_size]
    print('==TRAIN==')
    print_seq_split_stats(train_seq)
    x_train = np.array(list(map(lambda seq: seq[0], train_seq)))
    y_train = np.array(list(map(lambda seq: [seq[1]], train_seq)))
    
    print()
    
    test_seq = seq_data[-test_size:]
    print('==TEST==')
    print_seq_split_stats(test_seq)
    x_test = np.array(list(map(lambda seq: seq[0], test_seq)))
    y_test = np.array(list(map(lambda seq: [seq[1]], test_seq)))
    
    return x_train, y_train, x_test, y_test

def print_seq_split_stats(seq_data):
    """
    Prints the amount of data in the sequence and the distribution
    of the target split.
    """
    total = len(seq_data)
    buys = sum(map(lambda seq: seq[1], seq_data))
    dont_buys = len(list(filter(lambda seq: seq[1]==0, seq_data)))
    print(f'Total: {total}')
    print(f'Buys: {buys} ({round(buys/total*100, 0)}%)')
    print(f'Dont buys: {dont_buys} ({round(dont_buys/total*100, 0)}%)')

# `crypto_df` consists of rows split by timestamp and crypto asset.
# For this modeling, want the data to be split by timestamp only so that
#   each row contains data for each crypto asset that can be used as
#   `features` for the model.
df = crypto_df.head(400000).groupby(['timestamp', 'Symbol']).max().unstack()
df = df[['Volume', 'Close']]
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df.sort_index(inplace=True)

# Creating `target` column
df['Future_BTC'] = df['Close_BTC'].shift(-FUTURE_PERIOD_PREDICT)
df['Buy_BTC'] = df.apply(lambda row: classify(row.Close_BTC, row.Future_BTC), axis=1)
display(df[['Close_BTC', 'Future_BTC', 'Buy_BTC']])

# Preprocessing raw data into ML ready inputs
x_train, y_train, x_test, y_test = preprocess(df)


,Close_BTC,Future_BTC,Buy_BTC
timestamp,,,
1514764860,13850.176000,13768.040000,0
1514764920,13828.102000,13724.914000,0
1514764980,13801.314000,13717.112000,0
1514765040,13768.040000,13670.940000,0
1514765100,13724.914000,13721.348000,0
...,...,...,...
1517812020,8254.815000,8239.961667,0
1517812080,8261.260000,8201.318333,0
1517812140,8243.286667,NaN,0


==TRAIN==
Total: 45656
Buys: 22857 (50.0%)
Dont buys: 22799 (50.0%)

==TEST==
Total: 5073
Buys: 2443 (48.0%)
Dont buys: 2630 (52.0%)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Setting parameters
EPOCHS = 10
BATCH_SIZE = 64
NAME = f'{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}'

# Building RNN Model
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(x_train.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())
          
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

# Compiling RNN Model
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Setting up tensorbaord
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

filepath = 'RNN_Final-{epoch:02d}-{accuracy:.3f}'
checkpoint = ModelCheckpoint(
    'models/{}.model'.format(
        filepath, 
        monitor='val_acc', 
        verbose=1, 
        save_best_only=True, 
        mode='max')
)

history = model.fit(
    x_train, 
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard, checkpoint]
)

Epoch 1/10
714/714 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.7253

INFO:tensorflow:Assets written to: models/RNN_Final-01-0.725.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-01-0.725.model/assets


714/714 [==============================] - 335s 459ms/step - loss: 0.5415 - accuracy: 0.7253 - val_loss: 0.4145 - val_accuracy: 0.8082
Epoch 2/10
714/714 [==============================] - ETA: 0s - loss: 0.4167 - accuracy: 0.8107

INFO:tensorflow:Assets written to: models/RNN_Final-02-0.811.model/assets


...

INFO:tensorflow:Assets written to: models/RNN_Final-09-0.833.model/assets


714/714 [==============================] - 347s 486ms/step - loss: 0.3728 - accuracy: 0.8329 - val_loss: 0.3793 - val_accuracy: 0.8301
Epoch 10/10
714/714 [==============================] - ETA: 0s - loss: 0.3678 - accuracy: 0.8366

INFO:tensorflow:Assets written to: models/RNN_Final-10-0.837.model/assets


INFO:tensorflow:Assets written to: models/RNN_Final-10-0.837.model/assets


714/714 [==============================] - 357s 500ms/step - loss: 0.3678 - accuracy: 0.8366 - val_loss: 0.3814 - val_accuracy: 0.8315
